In [ ]:
from glob import glob
import numpy as np
import pandas as pd
from pathlib import Path
import sys
from itertools import combinations, product
sys.path.append('../')

import matplotlib.pyplot as plt
from tqdm import tqdm
from astropy.table import Table, unique, join
import papermill as pm
from astropy.time import Time
corr = __import__('11_corr')


In [ ]:
from uvot import filters
from xrt import curves
from source_names_dict import source_names_dict, source_names_w_counterparts, source_names_readable
from get_closest_srcreg import get_src_region_dict
from uvot import read_uvotsource, calc_uvot_flags, filters, filter_colors, filter_markers
from xrt import load_xrt, rename_xrt_cols, curves
from lightcurve import LightCurve

In [ ]:
def line(x, m, c):
    return m*x+c

In [ ]:
def load_df_prop(simbad_name, xrt_curve, uvot_filter, include_bad, include_UL):
    df = df_prop[(df_prop['simbad_name'] == simbad_name)
               & (df_prop['xrt_curve']   == xrt_curve)
               & (df_prop['uvot_filter'] == uvot_filter)
               & (df_prop['include_bad'] == include_bad)
               & (df_prop['include_UL']  == include_UL)]
    return df_prop

def load_corr_output(simbad_name, xrt_curve, uvot_filter, include_bad, include_UL):
    # Fix glob square bracket issue
    to_replace = {'[':'[[]',
                  ']':'[]]'}

    simbad_name_glob = simbad_name.translate(str.maketrans(to_replace))
    corr_output = glob(f'../lightcurves/correlation_output/*/tables/*{simbad_name_glob}*{xrt_curve}*{uvot_filter}*{include_bad}*{include_UL}*table_corr*')[0]
    
    df = pd.read_csv(corr_output)
    df['simbad_name'] = simbad_name
    df['xrt_curve']   = xrt_curve
    df['uvot_filter'] = uvot_filter
    df['include_bad'] = include_bad
    df['include_UL']  = include_UL
    return df

In [ ]:
def plot_corr(simbad_name, xrt_curve, uvot_filter, include_bad, include_UL):
    
    # Savefile
    fn = f'../lightcurves/correlation_output/{simbad_name}/plots/{simbad_name},{xrt_curve},{uvot_filter},{include_bad},{include_UL}'
    if Path(f'{fn}.pdf').exists():
        print(f'{fn} exists! skipping...')
        return None
    
    ###############
    ## LOAD DATA ##
    ###############    
    df_prop     = load_df_prop(simbad_name, xrt_curve, uvot_filter, include_bad, include_UL)
    df_corr_out = load_corr_output(simbad_name, xrt_curve, uvot_filter, include_bad, include_UL)
    
    to_replace = {'[':'[[]',
                  ']':'[]]'}
    simbad_name_glob = simbad_name.translate(str.maketrans(to_replace))
    
    uvot_rate     = 'COI_SRC_RATE'
    uvot_rate_err = 'COI_SRC_RATE_ERR'
    xrt_rate      = 'Rate'
    
    if xrt_curve == 'FULL':
        fits_path = f'../lightcurves/joined/{simbad_name},{uvot_filter},curve_nosys_join.fits'
        xrt_rate_err = 'Ratepos'
    else:
        fits_path = f'../lightcurves/joined/{simbad_name},{uvot_filter},hardrat_join,{xrt_curve}.fits'
        xrt_rate_err = 'Error'
        
    tab = Table.read(fits_path)
    tables = corr.calc_subsets(tab)
    tab_5_sig        = tables['tab_5_sig']
    tab_UL           = tables['tab_UL']
    tab_no_UL        = tables['tab_no_UL']
    tab_BAD          = tables['tab_BAD']
    tab_no_BAD       = tables['tab_no_BAD']
    tab_UL_no_bad    = tables['tab_UL_no_bad']
    tab_no_UL_no_bad = tables['tab_no_UL_no_bad']

    
    ###############
    ## Plot Data ##
    ###############
    
    width = 15
    height = width * 5/3

    # Set up axis
    fig = plt.figure(figsize=(width,height), facecolor='white')

    # Plot Sequential XRT Curve
    ax_lc1 = fig.add_subplot(511)
    ax_lc1.errorbar(tab_no_UL['idx'], tab_no_UL[xrt_rate], yerr=tab_no_UL[xrt_rate_err], color='black', lw=1.0, capsize=1.0, marker='x', ls='none')
    ax_lc1.errorbar(tab_UL['idx'], tab_UL[xrt_rate], yerr=[tab_UL[xrt_rate], [0]*len(tab_UL)], color='red', lw=1.0, capsize=1.0, marker='v', ls='none')
    ax_lc1.set_ylabel(f'{curve_energies[xrt_curve]} keV Rate (ct/s)')
    ax_lc1.set_xlabel('# Datapoint')
    ax_lc1.set_title(f'{source_names_readable[simbad_name]} | {uvot_filter} | {xrt_curve} | include_UL={include_UL} | include_bad={include_bad}')

    ax_lc2 = fig.add_subplot(512)
    ax_lc2.errorbar(range(len(tab)), tab[uvot_rate], yerr=tab[uvot_rate_err], color='purple', lw=1.0, capsize=1.0, marker='.', ls='none')
    ax_lc2.set_ylabel(f'{uvot_filter} Count Rate (ct/s)')
    ax_lc2.set_xlabel('# Datapoint')

    ax_lc3 = fig.add_subplot(513)

    t0 = Time(tab['MJD_1'].min(), format='mjd')
    ax_lc3.errorbar(tab_no_UL['MJD_0'], tab_no_UL[xrt_rate], yerr=tab_no_UL[xrt_rate_err], color='black', lw=1.0, capsize=1.0, marker='x', ls='none')
    ax_lc3.errorbar(tab_UL['MJD_0'], tab_UL[xrt_rate], yerr=[tab_UL[xrt_rate], [0]*len(tab_UL)], color='red', lw=1.0, capsize=1.0, marker='v', ls='none')

    ax_lc3_2 = ax_lc3.twinx()
    ax_lc3_2.errorbar(tab['MJD_0'], tab[uvot_rate], yerr=tab[uvot_rate_err], color='purple', lw=1.0, capsize=1.0, marker='.', ls='none')

    ax_lc3.set_xlabel(f'Time since {str(t0.datetime.date())} (days)')
    ax_lc3.set_ylabel(f'{curve_energies[xrt_curve]} keV Rate (ct/s)')
    ax_lc3_2.set_ylabel(f'{uvot_filter} Count Rate (ct/s)')

    #ax_lc3.errorbar(tab[xrt_rate], tab_UL[xrt_rate], yerr=[tab_UL[xrt_rate], [0]*len(tab_UL)], color='black', lw=1.0, capsize=1.0, marker='v', ls='none')


    # Plot bestfits
    def plot_bestfit(ax):
        xmin, xmax = ax.get_xlim()[0], ax.get_xlim()[1]
        xrange = np.linspace(xmin,xmax,100)
        ystds = np.array([line(x, df_corr_out['m'], df_corr_out['c']).std() for x in xrange])
        m_mu = df_corr_out['m'].mean()
        c_mu = df_corr_out['c'].mean()
        y_est = line(xrange, m_mu, c_mu)

        ax.plot(xrange, y_est, color='red')
        ax.fill_between(xrange, y_est - ystds, y_est + ystds, alpha=0.5, color='grey', ec='#5a5a5a')
        ax.fill_between(xrange, y_est - 2*ystds, y_est + 2*ystds, alpha=0.5, color='grey', ec='#5a5a5a')
        ax.set_xlim(xmin, xmax)


    ax_corr1 = fig.add_subplot(5,3,10)
    ax_corr1.errorbar(x=tab_UL[xrt_rate], y=tab_UL[uvot_rate], xerr=[tab_UL[xrt_rate], [0]*len(tab_UL)], yerr=tab_UL[uvot_rate_err], ls='none', lw=1.0, color='red', marker='<')
    ax_corr1.errorbar(x=tab_no_UL[xrt_rate], y=tab_no_UL[uvot_rate], xerr=tab_no_UL[xrt_rate_err], yerr=tab_no_UL[uvot_rate_err], ls='none', lw=1.0, color='black')
    ax_corr1.set_xlabel(f'{curve_energies[xrt_curve]} keV Rate (ct/s)')
    ax_corr1.set_ylabel(f'{uvot_filter} Count Rate (ct/s)')

    plot_bestfit(ax_corr1)



    ax_corr2 = fig.add_subplot(5,3,12)
    ax_corr2.scatter(x=tab_UL[xrt_rate], y=tab_UL[uvot_rate], color='red', marker='<')
    ax_corr2.scatter(x=tab_no_UL[xrt_rate], y=tab_no_UL[uvot_rate], color='black', marker='x')

    ax_corr2.set_xlabel(f'{curve_energies[xrt_curve]} keV Rate (ct/s)')
    ax_corr2.set_ylabel(f'{uvot_filter} Count Rate (ct/s)')

    plot_bestfit(ax_corr2)


    def plt_hist(ax, df, val):
        mu  = np.mean(df[val])
        std = np.std(df[val])
        ax.hist(df[val], bins=100, histtype='step', color='black')
        ax.axvline(mu, color='red', alpha=0.8, label=f'{mu:.2f} $\pm$ {std:.2f}')  
        ax.axvline(mu+std,  ls='--', color='grey', alpha=0.8)
        ax.axvline(mu-std,  ls='--', color='grey', alpha=0.8)
        ax.legend()


    ax_dist1 = fig.add_subplot(5,3,13)
    plt_hist(ax_dist1, df_corr_out, 'r')
    ax_dist1.set_xlim(-1, 1)
    ax_dist1.set_xlabel('r')

    ax_dist2 = fig.add_subplot(5,3,14)
    plt_hist(ax_dist2, df_corr_out, 'm')
    ax_dist2.set_xlabel('m')

    ax_dist3 = fig.add_subplot(5,3,15)
    plt_hist(ax_dist3, df_corr_out, 'c')
    ax_dist3.set_xlabel('c')

    
    print(f'Saving file to: {fn}')
    plt.savefig(f'{fn}.pdf', bbox_inches='tight')
    plt.savefig(f'{fn}.png', bbox_inches='tight')
    plt.show()
    return fig


    

In [ ]:
curve_energies = {'FULL' : '0.3 - 10.0',
                  'SOFT' : '0.3 - 1.5',
                  'HARD' : '1.5 - 10.0',
                  'HR'   : r'Ratio $\frac{(1.5 - 10.0)}{(0.3 - 1.5)}$'}

In [ ]:
all_df = []
for f in tqdm(glob('../lightcurves/correlation_output/*/tables/*table_prop*')):
    sp = f.split('/')[-1].split(',')
    simbad_name = sp[0]
    xrt_curve   = sp[1]
    uvot_filter = sp[2]
    include_bad = sp[3]
    include_UL  = sp[4]
    
    df = pd.read_csv(f)
    df['simbad_name'] = simbad_name
    df['xrt_curve'] = xrt_curve
    df['uvot_filter'] = uvot_filter
    df['include_bad'] = include_bad
    df['include_UL']  = include_UL
    
    all_df.append(df)
df_prop = pd.concat(all_df)
df_prop


In [ ]:
all_df = []
for f in tqdm(glob('../lightcurves/correlation_output/*/tables/*table_prop*')):
    sp = f.split('/')[-1].split(',')
    simbad_name = sp[0]
    xrt_curve   = sp[1]
    uvot_filter = sp[2]
    include_bad = sp[3]
    include_UL  = sp[4]
    print(sp)
    plot_corr(simbad_name=simbad_name, xrt_curve=xrt_curve, uvot_filter=uvot_filter, include_bad=include_bad, include_UL=include_UL)

In [ ]:
tab = Table.read('../lightcurves/joined/Holmberg_II_X-1,U,curve_nosys_join.fits')
mjd_min = 59274 # mar 2021
tab = tab[tab['MJD_2'] > 59274]

In [ ]:
tab[['MJD_2', 'COI_SRC_RATE', 'COI_SRC_RATE_ERR', 'NSIGMA', 'Rate', 'Ratepos', 'BAD', 'UL']]
#.write('HoII_MJD_59274_XRT_UVOT.dat', format='ascii', overwrite=True)

In [ ]:
tab

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
ax2 = ax.twinx()
ax.errorbar(tab['MJD_2'], tab['COI_SRC_RATE'], yerr=tab['COI_SRC_RATE_ERR'], ls='none', color='violet', lw=1.0, capsize=1.0, marker='.')
ax2.errorbar(tab['MJD_2'], tab['Rate'], yerr=tab['Ratepos'], ls='none', color='black', lw=1.0, capsize=1.0, marker='.')
ax.set_ylabel('XRT RATE')
ax.set_ylabel('UVOT UVM2 RATE')
